# IBM Qiskit Circuit on Dynex: 4 Qubit Full Adder 

In [1]:
from dynexsdk.qiskit import QuantumRegister, ClassicalRegister
from dynexsdk.qiskit import QuantumCircuit, execute

In [2]:
n = 4
a = QuantumRegister(n)
b = QuantumRegister(n)

C = QuantumRegister(n)
S = QuantumRegister(n)

ac = ClassicalRegister(n)
bc = ClassicalRegister(n)
Cc = ClassicalRegister(n)
Sc = ClassicalRegister(n)

circuit = QuantumCircuit(a, b, C, S, ac, bc, Cc, Sc)

circuit.ccx(a[n-1], b[n-1], C[n-1])
circuit.cx(a[n-1], S[n-1])
circuit.cx(b[n-1], S[n-1])

for i in range(1,n):
    circuit.ccx(a[n-1-i],b[n-1-i],C[n-1-i])
    circuit.ccx(b[n-1-i],C[n-1-(i-1)],C[n-1-i])
    circuit.ccx(a[n-1-i],C[n-1-(i-1)],C[n-1-i])
    circuit.cx(a[n-1-i], S[n-1-i])
    circuit.cx(b[n-1-i], S[n-1-i])
    circuit.cx(C[n-1-(i-1)], S[n-1-i])
    
circuit.measure(C, Cc)
circuit.measure(S, Sc)

#circuit.draw(output='mpl', scale=0.6, fold=-1)

IBM Qiskit on Dynex Neuromorphic Platform
[DYNEX] Dynex SDK version: 0.1.5
[DYNEX] Annealer Graph initialised


In [3]:
qubits_measured, qubits_unmeasured = execute(
    circuit, 
    sampler='dynex', 
    mainnet=True, 
    num_reads=50000, 
    annealing_time=1000, 
    logging=False, 
    constrain_measured_qubits=True,
    constraint_strength = 10
)

[DYNEX] Qiskit circuit qubits: 17
[DYNEX] Embedding has been built:
	number of qubits:  310
	number of couplers:  419
	number of non-gate couplers: 201
[DYNEX] INFO: q2_0 was constrained to be 0 (contrain_measured_qubits=True)
[DYNEX] INFO: q2_1 was constrained to be 0 (contrain_measured_qubits=True)
[DYNEX] INFO: q2_2 was constrained to be 0 (contrain_measured_qubits=True)
[DYNEX] INFO: q2_3 was constrained to be 0 (contrain_measured_qubits=True)
[DYNEX] INFO: q3_0 was constrained to be 0 (contrain_measured_qubits=True)
[DYNEX] INFO: q3_1 was constrained to be 0 (contrain_measured_qubits=True)
[DYNEX] INFO: q3_2 was constrained to be 0 (contrain_measured_qubits=True)
[DYNEX] INFO: q3_3 was constrained to be 0 (contrain_measured_qubits=True)
[ÐYNEX] BQM gnerated
	number of variables: 310
	number of qubit biases: 310
	number of coupler_strengths: 419
[DYNEX] Running Qiskit Circuit on the Dynex Neuromorphic Platform...
[DYNEX] Sampling completed. Result:
  32 35 36 39 44 47 52 55 60 63 6

In [4]:
print('measured:',qubits_measured)
print('unmeasured:',qubits_unmeasured)

measured: {'q2_0': 0, 'q2_1': 0, 'q2_2': 0, 'q2_3': 0, 'q3_0': 0, 'q3_1': 0, 'q3_2': 0, 'q3_3': 0}
unmeasured: {'q0_0': 0, 'q0_1': 0, 'q0_2': 0, 'q0_3': 0, 'q1_0': 0, 'q1_1': 0, 'q1_2': 0, 'q1_3': 0}


## verify correctness

In [5]:
# sum (first n qubits of measured)
cnt = 0;
bin_sum = "";
for q in qubits_measured:
    if cnt < n:
        bin_sum += str(qubits_measured[q])
    cnt += 1

# input A (first n qubits of unmeasured)
# input B (last n qubits of unmeasured)
cnt = 0;
bin_a = "";
bin_b = "";
for q in qubits_measured:
    if cnt < n:
        bin_a += str(qubits_measured[q])
    else:
        bin_b += str(qubits_measured[q])
    cnt += 1

In [6]:
# Calculating binary sum by using bin() and int()
binary_sum = lambda a,b : bin(int(a, 2) + int(b, 2))
get_bin = lambda x, n: format(x, 'b').zfill(n)
bin_sum_calc = get_bin(int(binary_sum(bin_a, bin_b)[2:]),n)

In [7]:
print('A   =', bin_a);
print('B   =', bin_b);
print('SUM =', bin_sum)
print('Correct result?',bin_sum == bin_sum_calc)

A   = 0000
B   = 0000
SUM = 0000
Correct result? True
